# 반도체 박막 두께 분석

### 배경

최근 고사양 반도체 수요가 많아지면서 반도체를 수직으로 적층하는 3차원 공정이 많이 연구되고 있습니다. 반도체 박막을 수십 ~ 수백 층 쌓아 올리는 공정에서는 박막의 결함으로 인한 두께와 균일도가 저하되는 문제가 있습니다. 이는 소자 구조의 변형을 야기하며 성능 하락의 주요 요인이 됩니다. 이를 사전에 방지하기 위해서는 박막의 두께를 빠르면서도 정확히 측정하는 것이 중요합니다.

박막의 두께를 측정하기 위해 광스펙트럼 분석이 널리 사용되고 있습니다. 하지만 광 스펙트럼을 분석하기 위해서는 관련 지식을 많이 가진 전문가가 필요하며 분석과정에 많은 컴퓨팅자원이 필요합니다. 빅데이터 분석을 통해 이를 해결하고자 반도체 소자의 두께 분석 알고리즘 경진대회를 개최합니다.



### 배경 자료

반도체 박막은 얇은 반도체 막으로 박막의 종류와 두께는 반도체 소자의 특성을 결정짓는 중요한 요소 중 하나입니다. 박막의 두께를 측정하는 방법으로 반사율 측정이 널리 사용되며 반사율은 입사광 세기에 대한 반사광 세기의 비율로 정해집니다. (반사율 = 반사광/입사광) 반사율은 빛의 파장에 따라 변하며 파장에 따른 반사율의 분포를 반사율 스펙트럼이라고 합니다.



### 구조 설명

이번 대회에서 분석할 소자는 질화규소(layer_1)/이산화규소(layer_2)/질화규소(layer_3)/이산화규소(layer_4)/규소(기판) 총 5층 구조로 되어 있습니다. 대회의 목적은 기판인 규소를 제외한 layer_1 ~ layer_4의 두께를 예측하는 것으로 학습 데이터 파일에는 각 층의 두께와 반사율 스펙트럼이 포함되어 있습니다.



### 데이터 설명

train.csv 파일에는 4층 박막의 두께와 파장에 따른 반사율 스펙트럼이 주어집니다. 헤더의 이름에 따라 layer_1 ~ 4는 해당 박막의 두께,

데이터값인 0 ~ 225은 빛의 파장에 해당하는 반사율이 됩니다. 헤더 이름인 0~225은 파장을 뜻하며 비식별화 처리가 되어있어 실제 값과는 다릅니다.

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Conv2D, SeparableConv2D, Flatten, Dropout, MaxPooling2D, \
    BatchNormalization, ReLU, GlobalAveragePooling2D
from tensorflow.keras import regularizers

In [2]:
use_colab = True
assert use_colab in [True, False]

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

### 데이터 로드
* 새로운 버전의 Colab파일에선 왼쪽의 폴더 tree에서 직접 드라이브 마운트를 진행해야합니다.

In [4]:
train_d = np.load("./data/semicon_train_data.npy")
test_d = np.load("./data/semicon_test_data.npy")

# train data path
# train_d = np.load("./semicon_train_data.npy")

# test data path
# train_d = np.load("./semicon_test_data.npy")
print(train_d.shape, test_d.shape)

(729000, 230) (81000, 230)


In [5]:
print(train_d[1].shape)

(230,)


In [6]:
print(test_d[0].shape)

(230,)


In [7]:
# Raw Data 확인
print(train_d[1000])
# print(test_d[0])

[7.00000000e+01 4.00000000e+01 1.50000000e+02 1.10000000e+02
 4.02068880e-01 4.05173400e-01 4.22107550e-01 4.35197980e-01
 4.66261860e-01 4.67785980e-01 4.76300870e-01 5.07203700e-01
 5.23296600e-01 5.31541170e-01 5.23319200e-01 5.32340650e-01
 5.38513840e-01 5.62478700e-01 5.74338900e-01 5.62083540e-01
 5.83467800e-01 5.88462350e-01 5.89606940e-01 5.93186560e-01
 5.93241930e-01 5.98241150e-01 5.87171000e-01 5.92055500e-01
 6.06527860e-01 5.87087870e-01 5.94492100e-01 5.73233500e-01
 5.94926300e-01 5.76777600e-01 5.70286300e-01 5.76913000e-01
 5.52754400e-01 5.61735150e-01 5.52500000e-01 5.23800200e-01
 5.11056840e-01 5.11732600e-01 4.92525460e-01 4.73373200e-01
 4.57444160e-01 4.53428800e-01 4.19270130e-01 3.87709920e-01
 3.87221520e-01 3.49848450e-01 3.16229370e-01 3.13858700e-01
 2.80085700e-01 2.61263000e-01 2.30250820e-01 1.98457850e-01
 1.53500440e-01 1.36026740e-01 1.08726785e-01 8.95344000e-02
 6.58975300e-02 5.09539300e-02 3.69604570e-02 2.88708470e-02
 1.54857090e-02 1.995940

* 데이터를 분석하여 모델에 학습할 수 있는 형태로 정리합니다.

In [8]:
train = train_d[:,4:] # 729000, 230 4번째 데이터부터 가져온다.
label = train_d[:,:4]

t_train = test_d[:,4:] # 81000, 230개 4번째 까지 데이터를 가져온다.
t_label = test_d[:,:4]

In [9]:
batch_size = 256

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train, label))
train_dataset = train_dataset.shuffle(10000).repeat().batch(batch_size=batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((t_train, t_label))
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)

<BatchDataset element_spec=(TensorSpec(shape=(None, 226), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(None, 226), dtype=tf.float64, name=None), TensorSpec(shape=(None, 4), dtype=tf.float64, name=None))>


### 모델 구성
* 현재 가장 간단한 모델로 구성되어 있습니다.
* 데이터셋 자체를 가장 잘 학습할 수 있는 모델을 구현해 학습을 진행합니다.

In [43]:
from tensorflow.keras import layers, regularizers, models
import tensorflow as tf

# 클래스버젼 - Seperable Conv2D
class ResBlock_without_RNN(tf.keras.Model):
    def __init__(self, num_filter, stride=1, kernel_size=3, l2_reg=1e-4, dim = None):
        super(ResBlock_without_RNN, self).__init__()
        self.num_filter = num_filter
        self.dim = dim
        # print(self.dim,self.num_filter)

        self.conv1 = layers.Conv2D(num_filter, kernel_size, strides=stride, padding='same',
                                   kernel_initializer=tf.keras.initializers.he_normal(),
                                   kernel_regularizer=regularizers.l2(0.001))
        self.se_conv1 = layers.SeparableConv2D(num_filter, kernel_size, padding='same',
                                               depthwise_initializer=tf.keras.initializers.he_normal(),
                                               pointwise_initializer=tf.keras.initializers.he_normal(),
                                               depthwise_regularizer=regularizers.l2(0.001),
                                               pointwise_regularizer=regularizers.l2(0.001))
        self.se_sonv2 = layers.SeparableConv2D(num_filter, kernel_size, padding='same',
                                               depthwise_initializer=tf.keras.initializers.he_normal(),
                                               pointwise_initializer=tf.keras.initializers.he_normal(),
                                               depthwise_regularizer=regularizers.l2(0.001),
                                               pointwise_regularizer=regularizers.l2(0.001))
        self.se_conv1d_1 = layers.SeparableConv1D(num_filter, kernel_size, padding='same',
                                                  depthwise_initializer=tf.keras.initializers.he_normal(),
                                                  pointwise_initializer=tf.keras.initializers.he_normal(),
                                                  depthwise_regularizer=regularizers.l2(0.001),
                                                  pointwise_regularizer=regularizers.l2(0.001))
        self.se_sonv1d_2 = layers.SeparableConv1D(num_filter, kernel_size, padding='same',
                                              depthwise_initializer=tf.keras.initializers.he_normal(),
                                              pointwise_initializer=tf.keras.initializers.he_normal(),
                                              depthwise_regularizer=regularizers.l2(0.001),
                                              pointwise_regularizer=regularizers.l2(0.001))
        self.con_short = layers.Conv2D(num_filter, 1, strides=2, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.bn3 = layers.BatchNormalization()
        self.bn_short = layers.BatchNormalization()
        # self.LSTM = tf.keras.layers.LSTM(dim*dim,return_sequences=True,recurrent_initializer='glorot_uniform')
        # self.LSTM2 = tf.keras.layers.LSTM(dim*dim,return_sequences=True,recurrent_initializer='glorot_uniform')
        # self.LSTM3 = tf.keras.layers.LSTM(dim*dim)
        # # self.con1d_1 = layers.Conv2D()

    def call(self, input, stride=1, dim=None):
        self.dim = dim
        shortcut = input
        x = self.conv1(input)

        x = self.bn1(x)
        x = layers.Activation('relu')(x)


        # print("변환하기전:",x.shape, dim, self.num_filter,stride)
        # RNN_x = layers.Reshape(target_shape=(-1, dim * dim * self.num_filter))(x) # 컨볼루션을 쓰기위한 변환 (64,4) -> CNN1D또는 RNN 사용가능 256,1도 가능
        # # print("변환후:",RNN_x.shape)
        # 
        # RNN_x = self.LSTM(RNN_x)
        # RNN_x = self.LSTM2(RNN_x)
        # RNN_x = self.LSTM3(RNN_x)
        # # print("LSTM후:",RNN_x.shape)
        # RNN_x = layers.Reshape(target_shape=(dim,dim,1))(RNN_x)  # RNN_x 리쉐이프
        # # print("재변환후:",RNN_x.shape)
        # C1D_x = RNN_x


        x = self.se_conv1(x)
        x = self.bn2(x)
        x = self.se_sonv2(x)
        x = self.bn3(x)

        # 입력과 출력의 맞도록 숏컷을 조절
        if shortcut.shape[-1] != self.num_filter or stride != 1:
            shortcut = self.con_short(shortcut)
            shortcut = self.bn_short(shortcut)

        x = layers.Add()([x, shortcut])
        # print("add후:",x.shape)
        x = layers.Activation('relu')(x)
        return x

class ResBlock(tf.keras.Model):
    def __init__(self, num_filter, stride=1, kernel_size=3, l2_reg=1e-4, dim = None):
        super(ResBlock, self).__init__()
        self.num_filter = num_filter
        self.dim = dim
        # print(self.dim,self.num_filter)

        self.conv1 = layers.Conv2D(num_filter, kernel_size, strides=stride, padding='same',
                                   kernel_initializer=tf.keras.initializers.he_normal(),
                                   kernel_regularizer=regularizers.l2(0.001))
        self.se_conv1 = layers.SeparableConv2D(num_filter, kernel_size, padding='same',
                                               depthwise_initializer=tf.keras.initializers.he_normal(),
                                               pointwise_initializer=tf.keras.initializers.he_normal(),
                                               depthwise_regularizer=regularizers.l2(0.001),
                                               pointwise_regularizer=regularizers.l2(0.001))
        self.se_sonv2 = layers.SeparableConv2D(num_filter, kernel_size, padding='same',
                                               depthwise_initializer=tf.keras.initializers.he_normal(),
                                               pointwise_initializer=tf.keras.initializers.he_normal(),
                                               depthwise_regularizer=regularizers.l2(0.001),
                                               pointwise_regularizer=regularizers.l2(0.001))
        self.se_conv1d_1 = layers.SeparableConv1D(num_filter, kernel_size, padding='same',
                                               depthwise_initializer=tf.keras.initializers.he_normal(),
                                               pointwise_initializer=tf.keras.initializers.he_normal(),
                                               depthwise_regularizer=regularizers.l2(0.001),
                                               pointwise_regularizer=regularizers.l2(0.001))
        self.se_sonv1d_2 = layers.SeparableConv1D(num_filter, kernel_size, padding='same',
                                           depthwise_initializer=tf.keras.initializers.he_normal(),
                                           pointwise_initializer=tf.keras.initializers.he_normal(),
                                           depthwise_regularizer=regularizers.l2(0.001),
                                           pointwise_regularizer=regularizers.l2(0.001))
        self.con_short = layers.Conv2D(num_filter, 1, strides=2, padding='same')
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.bn3 = layers.BatchNormalization()
        self.bn_short = layers.BatchNormalization()
        self.LSTM = tf.keras.layers.LSTM(dim*dim,return_sequences=True,recurrent_initializer='glorot_uniform')
        self.LSTM2 = tf.keras.layers.LSTM(dim*dim,return_sequences=True,recurrent_initializer='glorot_uniform')
        self.LSTM3 = tf.keras.layers.LSTM(dim*dim)
        # self.con1d_1 = layers.Conv2D()

    def call(self, input, stride=1, dim=None):
        self.dim = dim
        shortcut = input
        x = self.conv1(input)

        x = self.bn1(x)
        x = layers.Activation('relu')(x)


        # print("변환하기전:",x.shape, dim, self.num_filter,stride)
        RNN_x = layers.Reshape(target_shape=(-1, dim * dim * self.num_filter))(x) # 컨볼루션을 쓰기위한 변환 (64,4) -> CNN1D또는 RNN 사용가능 256,1도 가능
        # print("변환후:",RNN_x.shape)

        RNN_x = self.LSTM(RNN_x)
        RNN_x = self.LSTM2(RNN_x)
        RNN_x = self.LSTM3(RNN_x)
        # print("LSTM후:",RNN_x.shape)
        RNN_x = layers.Reshape(target_shape=(dim,dim,1))(RNN_x)  # RNN_x 리쉐이프
        # print("재변환후:",RNN_x.shape)
        # C1D_x = RNN_x


        x = self.se_conv1(x)
        x = self.bn2(x)
        x = self.se_sonv2(x)
        x = self.bn3(x)

        # 입력과 출력의 맞도록 숏컷을 조절
        if shortcut.shape[-1] != self.num_filter or stride != 1:
            shortcut = self.con_short(shortcut)
            shortcut = self.bn_short(shortcut)

        x = layers.Add()([x, shortcut,RNN_x])
        # print("add후:",x.shape)
        x = layers.Activation('relu')(x)
        return x

class ResNet28(tf.keras.Model):
    def __init__(self):
        super(ResNet28, self).__init__()
        self.dim = None  # dim 초기화

        self.conv1 = layers.Conv2D(64, 7, strides=2,
                                   kernel_initializer=tf.keras.initializers.he_normal(),
                                   padding='same')
        self.bn1 = layers.BatchNormalization()
        self.bn2 = layers.BatchNormalization()
        self.bn3 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()
        self.relu2 = layers.ReLU()

        self.blocks = [
            ResBlock_without_RNN(64, stride=1, dim = 8),
            ResBlock(128, stride=2, dim = 4),
            ResBlock(256, stride=2, dim = 2),
            ResBlock(256, stride=1, dim = 2),
            ResBlock(512, stride=2, dim = 1)
        ]

        self.avg_pool = layers.GlobalAveragePooling2D()
        self.dropout = layers.Dropout(0.5)
        self.out = layers.Dense(4)

    def call(self, inputs):
        x = layers.Dense(16 * 16 * 1)(inputs)
        x = layers.Reshape(target_shape=(16, 16, 1))(x)  # 컨볼루션을 쓰기위한 변환 (64,4) -> CNN1D또는 RNN 사용가능 256,1도 가능
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)  # 8x8*64


        x = self.blocks[0](x,dim =8)
        x = self.blocks[1](x,dim =4)
        x = self.blocks[2](x,dim =2)
        x = self.blocks[3](x,dim =2)
        x = self.blocks[4](x,dim =1)
        x = self.avg_pool(x)
        x = layers.Dense(1000)(x)
        # x =  x = layers.Reshape(target_shape=(1))(x)
        # x = layers.LSTM(64)(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.out(x)

        return x


# Specify input shape and number of classes
input_tensor = layers.Input(shape=(226,))  # Example input shape for image classification

# Build ResNet model
Res = ResNet28()
output_tensor = Res.call(input_tensor)
model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)

# Display model summary
model.summary()


Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 226)]             0         
                                                                 
 dense_23 (Dense)            (None, 256)               58112     
                                                                 
 reshape_8 (Reshape)         (None, 16, 16, 1)         0         
                                                                 
 conv2d_89 (Conv2D)          (None, 8, 8, 64)          3200      
                                                                 
 batch_normalization_187 (Ba  (None, 8, 8, 64)         256       
 tchNormalization)                                               
                                                                 
 re_lu_18 (ReLU)             (None, 8, 8, 64)          0         
                                                           

In [44]:
# # 수직연산

# from tensorflow.keras import layers, models

# input_tensor = layers.Input(shape=(226,))

# # 목표 출력의 형태가 (batch_size, 4)인 경우에 맞춤
# # target_shape = (16, 16, 4)

# x = layers.Dense(8 * 8 * 4, activation='relu')(input_tensor)

# x = layers.Reshape(target_shape=(16, 16, 1))(x)
# x = layers.Conv2D(64, 3, padding='same')(x)

# x = layers.Reshape(target_shape=(-1, 16 * 16 * 64))(x)
# x = layers.LSTM(256)(x)
# x = layers.Reshape(target_shape=(16, 16, 1))(x)


# x = layers.Conv2D(64, 3, padding='same')(x)
# x = layers.Flatten()(x)
# x = layers.Dense(64, activation='relu')(x)

# output_tensor = layers.Dense(4)(x)  # 목표 데이터 형태에 맞게 조정

# model = models.Model(input_tensor, output_tensor)
# model.summary()


In [45]:
# # 컨벌루션 하고 병렬연산

# from tensorflow.keras import layers, models

# input_tensor = layers.Input(shape=(226,))

# # 목표 출력의 형태가 (batch_size, 4)인 경우에 맞춤
# # target_shape = (16, 16, 4)

# x_input = layers.Dense(8 * 8 * 4, activation='relu')(input_tensor)

# x = layers.Reshape(target_shape=(16, 16, 1))(x_input)
# x_bi = layers.Conv2D(64, 3, padding='same',strides =2)(x)
# # 8 * 8* 64 사이즈가 됨

# x = layers.Conv2D(64, 3, padding='same')(x_bi)


# # 8* 8*64 사이즈를 계산해봄

# x2 = layers.Reshape(target_shape=(-1, 8 * 8*64))(x_bi)
# x2 = layers.LSTM(8*8*64)(x2)
# x2 = layers.Reshape(target_shape=(8, 8, 64))(x2)

# x = layers.add([x,x2])


# x = layers.Conv2D(64, 3, padding='same')(x)
# x = layers.Flatten()(x)
# x = layers.Dense(64, activation='relu')(x)

# output_tensor = layers.Dense(4)(x)  # 목표 데이터 형태에 맞게 조정

# model = models.Model(input_tensor, output_tensor)
# model.summary()


In [46]:
# # 컨벌루션 하고 병렬연산  - 작동하지만 사이즈가 너무 커짐

# from tensorflow.keras import layers, models

# input_tensor = layers.Input(shape=(226,))

# # 목표 출력의 형태가 (batch_size, 4)인 경우에 맞춤
# # target_shape = (16, 16, 4)

# x_input = layers.Dense(8 * 8 * 4, activation='relu')(input_tensor)

# x = layers.Reshape(target_shape=(16, 16, 1))(x_input)
# x_bi = layers.Conv2D(64, 3, padding='same',strides =2)(x)
# # 8 * 8* 64 사이즈가 됨

# x = layers.Conv2D(64, 3, padding='same')(x_bi)


# # 8* 8*64 사이즈를 계산해봄

# x2 = layers.Reshape(target_shape=(-1, 8 * 8*64))(x_bi)
# x2 = layers.LSTM(8*8*64)(x2)
# x2 = layers.Reshape(target_shape=(8, 8, 64))(x2)

# x = layers.add([x,x2])


# x = layers.Conv2D(64, 3, padding='same')(x)
# x = layers.Flatten()(x)
# x = layers.Dense(64, activation='relu')(x)

# output_tensor = layers.Dense(4)(x)  # 목표 데이터 형태에 맞게 조정

# model = models.Model(input_tensor, output_tensor)
# model.summary()


In [47]:
# # 컨벌루션 하고 병렬연산

# from tensorflow.keras import layers, models

# input_tensor = layers.Input(shape=(226,))

# # 목표 출력의 형태가 (batch_size, 4)인 경우에 맞춤
# # target_shape = (16, 16, 4)

# x_input = layers.Dense(8 * 8 * 4, activation='relu')(input_tensor)

# x = layers.Reshape(target_shape=(16, 16, 1))(x_input)
# x_bi = layers.Conv2D(64, 3, padding='same',strides =2)(x)
# # 8 * 8* 64 사이즈가 됨

# x = layers.Conv2D(64, 3, padding='same')(x_bi)


# # 8* 8*64 사이즈를 계산해봄

# x2 = layers.Reshape(target_shape=(-1, 8 * 8*64))(x_bi)
# x2 = layers.LSTM(8*8*64)(x2)
# x2 = layers.Reshape(target_shape=(8, 8, 64))(x2)

# x = layers.add([x,x2])


# x = layers.Conv2D(64, 3, padding='same')(x)
# x = layers.Flatten()(x)
# x = layers.Dense(64, activation='relu')(x)

# output_tensor = layers.Dense(4)(x)  # 목표 데이터 형태에 맞게 조정

# model = models.Model(input_tensor, output_tensor)
# model.summary()


In [48]:
# # 컨벌루션 하고 병렬연산  - 사이즈 조절해서 add
# from tensorflow.keras import layers, models

# input_tensor = layers.Input(shape=(226,))

# # 목표 출력의 형태가 (batch_size, 4)인 경우에 맞춤
# # target_shape = (16, 16, 4)

# x_input = layers.Dense(8 * 8 * 4, activation='relu')(input_tensor)

# x = layers.Reshape(target_shape=(16, 16, 1))(x_input)
# x_bi = layers.Conv2D(64, 3, padding='same',strides =2)(x)
# # 8 * 8* 64 사이즈가 됨

# x = layers.Conv2D(64, 3, padding='same')(x_bi)


# # 8* 8*64 사이즈를 계산해봄

# x2 = layers.Reshape(target_shape=(-1, 8 * 8*64))(x_bi)
# x2 = layers.LSTM(32, return_sequences=True)(x2)  # 여기서 사이즈를 줄이면 됨
# x2 = layers.LSTM(64)(x2) # 여기서 사이즈를 줄이면 됨
# x2 = layers.Reshape(target_shape=(8, 8, 1))(x2)

# x = layers.add([x,x2])


# x = layers.Conv2D(64, 3, padding='same')(x)
# x = layers.Flatten()(x)
# x = layers.Dense(64, activation='relu')(x)

# output_tensor = layers.Dense(4)(x)  # 목표 데이터 형태에 맞게 조정

# model = models.Model(input_tensor, output_tensor)
# model.summary()


In [49]:
# input_tensor = layers.Input(shape=(226,))
# # CNN1D, CNN2D, RNN 세가지를 사용해서 모델링

# # 226에서 256으로 확장
# x = layers.Dense(8*8*4, activation = 'relu')(input_tensor)

# CNN_x = layers.Reshape(target_shape=(8,8,4))(x) # 컨볼루션을 쓰기위한 변환 (64,4) -> CNN1D또는 RNN 사용가능 256,1도 가능
# RNN_x = layers.Reshape(target_shape=(256,1))(x)
# Dense_x = RNN_x


# x = tf.add([x, x_skip]) # batch, 128 + 226


# x = layers.Dense(16,
#                  kernel_initializer=tf.keras.initializers.HeUniform())(x)
# x = layers.BatchNormalization()(x)
# x = layers.Activation('relu')(x)

# output_tensor = layers.Dense(4)(x)

# model = tf.keras.Model(input_tensor, output_tensor)

### 모델 학습
* 4개층의 박막의 두께를 예측하는 모델을 학습해봅시다.

In [50]:
# the save point
if use_colab:
    checkpoint_dir ='./drive/My Drive/train_ckpt/semiconductor/exp1'
    if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)
else:
    checkpoint_dir = 'semiconductor/exp1'



In [51]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_mae',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=15,
                                                     monitor='val_loss',
                                                     restore_best_weights=True,

                                                     verbose=1)
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',  # 관찰할 지표
                              factor=0.2,  # 학습률을 줄이는 비율
                              patience=5,  # 몇 번의 에포크 동안 감소하지 않아야 하는지
                              min_lr=1e-9)

In [52]:
model.compile(loss='mae', #mse
              optimizer='adam',
              metrics=['mae']) #mse

In [53]:
history = model.fit(train_dataset,
                    steps_per_epoch=len(train) / batch_size, # train data의 길이 // batch 길이
                    epochs=10,
                    callbacks=[cp_callback,early_stopping_cb,reduce_lr],
                    validation_data=test_dataset,
                    validation_steps=len(t_train) / batch_size)

Epoch 1/10
2847/2847 [============================>.] - ETA: 0s - loss: 45.5541 - mae: 43.4169
Epoch 1: val_mae improved from inf to 22.46510, saving model to ./drive/My Drive/train_ckpt/semiconductor\exp1
2847/2847 [==============================] - 111s 34ms/step - loss: 45.5468 - mae: 43.4096 - val_loss: 24.6465 - val_mae: 22.4651 - lr: 0.0010
Epoch 2/10
2848/2847 [==============================] - ETA: 0s - loss: 19.4296 - mae: 17.1164
Epoch 2: val_mae improved from 22.46510 to 11.83725, saving model to ./drive/My Drive/train_ckpt/semiconductor\exp1
2847/2847 [==============================] - 98s 34ms/step - loss: 19.4296 - mae: 17.1164 - val_loss: 14.2244 - val_mae: 11.8373 - lr: 0.0010
Epoch 3/10
2847/2847 [============================>.] - ETA: 0s - loss: 16.0553 - mae: 13.6689
Epoch 3: val_mae improved from 11.83725 to 10.43833, saving model to ./drive/My Drive/train_ckpt/semiconductor\exp1
2847/2847 [==============================] - 99s 35ms/step - loss: 16.0552 - mae: 13.66

KeyboardInterrupt: 

In [ ]:
# np.save("file/path.npy", history.numpy())
# history = np.load("histoy.npy", allow_pickle=True)

loss=history.history['mae'] # mse
val_loss=history.history['val_mae'] # val_mse

epochs_range = range(len(loss))

plt.figure(figsize=(8, 4))
plt.plot(epochs_range, loss, label='Training MAE') # MSE
plt.plot(epochs_range, val_loss, label='Validation MAE') # MSE
plt.legend(loc='upper right')
plt.title('Training and Validation MAE') # MSE
plt.show()

### 모델 평가
* Mean absolute error 를 이용해 모델이 정확히 예측하는지를 확인

In [ ]:
model.load_weights(checkpoint_dir)

In [ ]:
# model.load_weights(checkpoint_dir)
results = model.evaluate(test_dataset)
print("MAE :", results[0]) # MAE